In [1]:
%load_ext autoreload
%autoreload 2

# TransducedLM vs FusedTransducedLM Benchmark (BPE)

Compares two approaches to computing next-byte log-probabilities through
a BPE FST (maps GPT-2 token IDs → bytes):

- **TransducedLM**: two-phase (PeekabooState BFS decomposition, then LM-weighted search)
- **FusedTransducedLM**: single-pass (interleaved decomposition + LM search, no separate BFS)

Uses a 3-gram CharNgramLM over token IDs as the inner LM, with per-call
timeouts and a process-wide memory limit.

The BPE FST has a star topology: each token maps to its byte spelling via
a chain of epsilon-input arcs, then a single token-consuming arc back to
the start state. This creates a very different decomposition structure
from the PTB tokenizer (which has complex rewrite rules and 296 states).

In [2]:
import time, gc
import numpy as np
from matplotlib import pyplot as plt
from transformers import AutoTokenizer
from transduction.applications.bpe import bpe_wfst
from transduction.fst import FST
from transduction.fsa import EPSILON
from transduction.lm.statelm import HfTokenizerVocab
from transduction.util import Timeout, timelimit, set_memory_limit
set_memory_limit(8)

In [3]:
# ---- Tunable parameters ----
VOCAB_SIZE = 1_000    # None → only tokens from training data (~43)
                       # int  → use that many from GPT-2 vocab (max ~50k)

In [4]:
tokenizer = AutoTokenizer.from_pretrained('gpt2', use_fast=False, local_files_only=True)
_decode = HfTokenizerVocab(tokenizer).decode
drop = {x.encode() for x in tokenizer.all_special_tokens}

# All non-special token IDs available
all_token_ids = sorted(i for i in range(len(_decode)) if _decode[i] not in drop)

# Training data for the inner n-gram LM
train_sentences = [
    "The quick brown fox jumps over the lazy dog.",
    "A stitch in time saves nine.",
    "To be or not to be, that is the question.",
    "All that glitters is not gold.",
    "Actions speak louder than words.",
    "Practice makes perfect.",
    "Where there is a will, there is a way.",
] * 3

train_ids = [tokenizer.encode(s) for s in train_sentences]
train_used = sorted(set(tid for seq in train_ids for tid in seq))

# Select vocabulary subset
if VOCAB_SIZE is None:
    used_ids = train_used
else:
    used_ids = all_token_ids[:VOCAB_SIZE]
    # Always include training tokens so n-gram LM has valid data
    used_ids = sorted(set(used_ids) | set(train_used))

print(f'Training tokens: {len(train_used)}, Selected vocab: {len(used_ids)} / {len(all_token_ids)} total')

# Build subsampled BPE FST
def subsampled_bpe_fst(decode, token_ids, drop=frozenset()):
    """Build a BPE FST from a subset of token IDs."""
    m = FST()
    m.add_start(())
    for i in token_ids:
        x = decode[i]
        if x in drop:
            continue
        bx = tuple(x)
        for j in range(len(bx)):
            m.add_arc(bx[:j], EPSILON, bx[j], bx[:j+1])
        m.add_arc(bx, i, EPSILON, ())
    m.add_stop(())
    return m

t0 = time.perf_counter()
bpe_fst = subsampled_bpe_fst(_decode, used_ids, drop)
print(f'BPE FST built in {time.perf_counter()-t0:.3f}s: '
      f'{len(bpe_fst.states)} states, |A|={len(bpe_fst.A)}, |B|={len(bpe_fst.B)}')

Training tokens: 43, Selected vocab: 1023 / 50256 total
BPE FST built in 0.009s: 1313 states, |A|=1024, |B|=257


In [5]:
# Generate target byte sequence by transducing token IDs through the FST
text = "The quick brown fox jumps over the lazy dog."
token_ids_test = tokenizer.encode(text)
target_seq = list(bpe_fst.transduce(token_ids_test))
print(f'Text: {text!r}')
print(f'Token IDs: {token_ids_test}')
print(f'Target (bytes): {len(target_seq)} symbols')
print(f'Decoded: {bytes(target_seq).decode()!r}')

# Train inner LM on token-ID sequences
from transduction.lm.ngram import CharNgramLM
source_alpha = bpe_fst.A - {EPSILON}
inner_lm = CharNgramLM.train(train_ids, n=3, alpha=0.5, alphabet=source_alpha)
print(f'Inner LM: alphabet={len(inner_lm.alphabet)} symbols')

Text: 'The quick brown fox jumps over the lazy dog.'
Token IDs: [464, 2068, 7586, 21831, 18045, 625, 262, 16931, 3290, 13]
Target (bytes): 44 symbols
Decoded: 'The quick brown fox jumps over the lazy dog.'
Inner LM: alphabet=1024 symbols


## TransducedLM Scaling

Per-step decode time for **TransducedLM** (two-phase: PeekabooState BFS
decomposition, then LM-weighted search) vs **FusedTransducedLM** (single-pass:
interleaved decomposition + LM search, no separate BFS).

Each step includes both decomposition and LM search costs. Both use
`K=20`, `max_expansions=200`, with a 3s timeout per step.

In [6]:
from collections import defaultdict
from transduction.lm.transduced import TransducedLM
from transduction.lm.fused_transduced import FusedTransducedLM
from transduction.token_decompose import TokenPeekabooState

MAX_DECODE = 100             # number of decode steps
MAX_SEARCH = 200             # max priority-queue steps per logp_next
MAX_BEAM = 10                # max items carried forward
LM_TIMEOUT = 10               # seconds per step

lm_results = defaultdict(list)  # name -> [(step, time_s, logp)]

configs = [
#    ('TransducedLM', lambda: TransducedLM(
#        inner_lm, bpe_fst, K=MAX_BEAM, max_expansions=MAX_SEARCH)),
#    ('TransducedLM (token)', lambda: TransducedLM(
#        inner_lm, bpe_fst, K=MAX_BEAM, max_expansions=MAX_SEARCH,
#        decomp_state_cls=TokenPeekabooState, univ_cls=lambda fst: None)),
    ('FusedTransducedLM', lambda: FusedTransducedLM(
        inner_lm, bpe_fst, max_steps=MAX_SEARCH, max_beam=MAX_BEAM)),
    ('FusedTransducedLM (token)', lambda: FusedTransducedLM(
        inner_lm, bpe_fst, max_steps=MAX_SEARCH, max_beam=MAX_BEAM, helper='token')),
]

for name, make_tlm in configs:
    print(f'\n{name} (K={MAX_BEAM}, max_expansions={MAX_SEARCH}):')
    try:
        with timelimit(LM_TIMEOUT):
            tlm = make_tlm()
            state = tlm.initial()
    except (Timeout, MemoryError, ValueError) as e:
        print(f'  initial() failed: {type(e).__name__}: {e}')
        continue
    for i in range(min(MAX_DECODE, len(target_seq))):
        y = target_seq[i]
        try:
            with timelimit(LM_TIMEOUT):
                t0 = time.perf_counter()
                lp = state.logp_next[y]
                state = state >> y
                t1 = time.perf_counter()
        except Timeout:
            print(f'  step {i+1} TIMEOUT ({LM_TIMEOUT}s)')
            break
        except MemoryError:
            print(f'  step {i+1} OOM')
            break
        elapsed = t1 - t0
        lm_results[name].append((i + 1, elapsed, lp))
        print(f'  {i+1:2d}: {elapsed*1000:8.1f} ms  logp={lp:.4f}')
    gc.collect()


FusedTransducedLM (K=10, max_expansions=200):
   1:     71.2 ms  logp=-4.4900
   2:     68.5 ms  logp=-0.6925
   3:     73.9 ms  logp=-0.1305
   4:     70.1 ms  logp=-1.0085
   5:     19.5 ms  logp=-0.8267
   6:      0.7 ms  logp=0.0000
   7:      0.4 ms  logp=0.0000
   8:      0.3 ms  logp=0.0000
   9:      0.3 ms  logp=0.0000
  10:     58.1 ms  logp=-1.0077
  11:     16.6 ms  logp=-0.8267
  12:      0.9 ms  logp=0.0000
  13:      0.4 ms  logp=0.0000
  14:      0.2 ms  logp=0.0000
  15:      0.3 ms  logp=0.0000
  16:     67.8 ms  logp=-1.0077
  17:     15.6 ms  logp=-0.6931
  18:      1.1 ms  logp=-0.1335
  19:      0.5 ms  logp=0.0000
  20:     61.2 ms  logp=-1.0077
  21:     16.8 ms  logp=-0.8267
  22:      0.7 ms  logp=0.0000
  23:      0.3 ms  logp=0.0000
  24:      0.3 ms  logp=0.0000
  25:      0.4 ms  logp=0.0000
  26:     56.5 ms  logp=-1.0077
  27:     15.8 ms  logp=-0.8267
  28:      1.0 ms  logp=0.0000
  29:      0.5 ms  logp=0.0000
  30:      0.3 ms  logp=0.0000
  31:    

KeyboardInterrupt: 

In [ ]:
# Summary table
print(f'\n{"Algorithm":<25s} {"Total (s)":>10s} {"Avg/step (ms)":>14s} {"Steps":>6s}')
print('-' * 57)
for name, data in sorted(lm_results.items()):
    total = sum(t for _, t, _ in data)
    avg = total / len(data) * 1000
    print(f'{name:<25s} {total:10.2f} {avg:14.1f} {len(data):6d}')

# Pairwise logp agreement
names = sorted(lm_results.keys())
if len(names) >= 2:
    ref_name = names[0]
    ref_data = lm_results[ref_name]
    print(f'\nMax |logp| diff vs {ref_name}:')
    for name in names[1:]:
        data = lm_results[name]
        n = min(len(ref_data), len(data))
        if n > 0:
            diffs = [abs(ref_data[i][2] - data[i][2]) for i in range(n)]
            print(f'  {name}: {max(diffs):.6f}')

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

fig, ax = plt.subplots(figsize=(10, 5))

for name, data in sorted(lm_results.items()):
    steps = [d[0] for d in data]
    times = [d[1] * 1000 for d in data]  # ms
    ax.plot(steps, times, 'o-', label=name, markersize=4)
ax.set_xlabel('Target step (byte index)')
ax.set_ylabel('Time per step (ms)')
ax.set_title(f'TransducedLM Variants (BPE, max_steps={MAX_SEARCH})')
ax.legend()
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## Vocab-Size Scaling

How does FusedTransducedLM scale as the BPE vocabulary grows?
At boundary DFA states, rho-arc compression stores a single default
destination instead of |V| explicit arcs. But with Strategy A (full
enumeration), we still push all |V| rho-class symbols into the priority
queue at each expansion. This section measures the wall.

The key metric is **avg ms/step at boundary steps** (where the target
byte aligns with a token boundary, triggering expansion over all source
symbols).

In [ ]:
import time, gc
import numpy as np
from collections import defaultdict
from transduction.fst import FST
from transduction.fsa import EPSILON
from transduction.lm.ngram import CharNgramLM
from transduction.lm.transduced import TransducedLM
from transduction.lm.fused_transduced import FusedTransducedLM
from transduction.token_decompose import TokenPeekabooState
from transduction.util import Timeout, timelimit

text_short = "The quick brown fox"
token_ids_short = tokenizer.encode(text_short)
target_bytes_short = list(bpe_fst.transduce(token_ids_short))[:8]

VOCAB_SIZES = [None, 200, 500, 1000]# 2000, 5000, 7000]
SCALE_TIMEOUT = 60   # seconds per vocab size

# ---- Methods to compare (name, factory(inner_lm, fst) -> TransducedLM) ----
methods = [
    ('FusedTransducedLM',       lambda lm, fst: FusedTransducedLM(lm, fst, max_steps=200, max_beam=10)),
    ('FusedTransducedLM (token)', lambda lm, fst: FusedTransducedLM(lm, fst, max_steps=200, max_beam=10, helper='token')),
    #('TransducedLM',            lambda lm, fst: TransducedLM(lm, fst, K=10, max_expansions=200)),
    #('TransducedLM (token)',    lambda lm, fst: TransducedLM(lm, fst, K=10, max_expansions=200,
    #                                decomp_state_cls=TokenPeekabooState, univ_cls=lambda fst: None)),
]

scale_results = defaultdict(list)  # method_name -> [(vocab_size, avg_ms, max_ms, total_ms, n_steps)]

for vs in VOCAB_SIZES:
    if vs is None:
        used = train_used
        label = f'{len(train_used)} (train)'
    else:
        used = sorted(set(all_token_ids[:vs]) | set(train_used))
        label = str(len(used))

    fst_v = subsampled_bpe_fst(_decode, used, drop)
    source_alpha_v = fst_v.A - {EPSILON}
    inner_v = CharNgramLM.train(train_ids, n=3, alpha=0.5, alphabet=source_alpha_v)

    for method_name, make_tlm in methods:
        tlm_v = make_tlm(inner_v, fst_v)
        step_times = []
        try:
            with timelimit(SCALE_TIMEOUT):
                state_v = tlm_v.initial()
                for yb in target_bytes_short:
                    t0 = time.perf_counter()
                    _ = state_v.logp_next[yb]
                    state_v = state_v >> yb
                    step_times.append(time.perf_counter() - t0)
        except (Timeout, MemoryError) as e:
            print(f'V={label:>6s} {method_name}: {type(e).__name__} after {len(step_times)} steps')
            if not step_times:
                continue

        avg = np.mean(step_times) * 1000
        mx = np.max(step_times) * 1000
        tot = np.sum(step_times) * 1000
        scale_results[method_name].append((len(used), avg, mx, tot, len(step_times)))
        print(f'V={label:>6s}  {method_name:<28s}  avg={avg:.0f}ms  max={mx:.0f}ms  total={tot:.0f}ms  ({len(step_times)} steps)')
        gc.collect()

V=43 (train)  FusedTransducedLM             avg=0ms  max=1ms  total=3ms  (8 steps)
V=43 (train)  FusedTransducedLM (token)     avg=6ms  max=17ms  total=46ms  (8 steps)
V=   240  FusedTransducedLM             avg=6ms  max=14ms  total=50ms  (8 steps)
V=   240  FusedTransducedLM (token)     avg=150ms  max=306ms  total=1199ms  (8 steps)
V=   529  FusedTransducedLM             avg=16ms  max=31ms  total=125ms  (8 steps)


In [ ]:
fig, ax = plt.subplots(figsize=(10, 5))
for method_name, data in sorted(scale_results.items()):
    vs_list = [r[0] for r in data]
    avgs = [r[1] for r in data]
    ax.plot(vs_list, avgs, 'o-', label=method_name, markersize=6)
ax.set_xlabel('Vocabulary size |V|')
ax.set_ylabel('Avg time per step (ms)')
ax.set_title('BPE Vocab Scaling: Method Comparison')
ax.legend()
ax.grid(True, alpha=0.3)
#ax.set_xscale('log')
#ax.set_yscale('log')
plt.tight_layout()
plt.show()